In [ ]:
import re
import requests
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/gdrive')

URL = 'https://sites.google.com/up.edu.mx/har-up/'
r = requests.get(URL)
soup = BeautifulSoup(r.content,  'lxml')
string = soup.prettify(formatter="minimal")
# regex = r'''div id=&quot;(Subject\d+Activity\d+Trial\d)&lt;/h5&gt;
# &lt;p&gt;
#     &lt;a href=&quot;((?!&quot;&)[\s\S]+)&quot;&gt;Camera1'''
regex = '&lt;h5&gt;(Subject\d+Activity\d+Trial\d)[\s\S]{718}([\s\S]{76})[\s\S]{70}([\s\S]{76})'
match = re.findall(regex, string)
print(len(match), 'matches found')
print('Example match:')
print(match[0])

561 matches found
('Subject1Activity1Trial1', 'https://drive.google.com/a/up.edu.mx/uc?id=1x-gpsGcP1jMAvWAZ9oM1O7MsGW8VaCma', 'https://drive.google.com/a/up.edu.mx/uc?id=1Lh9EMMbDu1Z8yooJVXc4FeAHrOFuR2NL')
('Subject1Activity1Trial2', 'https://drive.google.com/a/up.edu.mx/uc?id=1x8CipMrTCQl_Hy5HJpIO7TtmqYg4GC2K', 'https://drive.google.com/a/up.edu.mx/uc?id=1KnQGzijQbd90cv1s9XA4WEjI6cA_VDpq')


In [ ]:
! pip install -U --no-cache-dir gdown --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14951 sha256=cb041690e3b8d52f4df0457c67a5edc1af9ae668d894ec4a9b91f659914f1ae9
  Stored in directory: /tmp/pip-ephem-wheel-cache-7uwtdl1x/wheels/3d/ec/b0/a96d1d126183f98570a785e6bf8789fca559853a9260e928e1
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
# from pydrive.drive import GoogleDrive
import shutil
import gdown
import zipfile
import os

down = '/content/downloads/'
if not os.path.exists(down):
     os.makedirs(down, 0o700)

# you may have to download in batches since google drive can freeze downloads after a certain number of downloads
for case in match:
  name = down+case[0]
  for i in range(1,3):
    dir_name = name+f'{i}'
    gdown.download(case[i], dir_name+'.zip', quiet=False)
    zipf = dir_name + '.zip'
    n = case[i] + '&export=donwload'
    # ! gdown --no-cookies $n -O $zipf
    # download_file_from_google_drive((case[i].split('id='))[1], dir_name+'.zip')
    with zipfile.ZipFile(dir_name+'.zip', 'r') as zip_ref:
      os.mkdir(dir_name)
      zip_ref.extractall(dir_name)
    out = down+case[0]+f'Cam{i}'
    # generate video at 18 fps (-r 18) using ffmpeg
    ! ffmpeg -pattern_type glob -i '{dir_name}/*.png' -r 18 $out\.mp4
    # shutil.rmtree(dir_name)
    # os.remove(dir_name+'.zip')


In [ ]:
# files saved as /content/downloads/Subject1Activity1Trial2Cam1.mp4